In [4]:
from statsmodels.tsa.api import AutoReg, VAR
from stargazer.stargazer import Stargazer
from geopy.distance import great_circle
import matplotlib.pyplot as plt
import sklearn.metrics as skm
from tqdm import tqdm
import pandas as pd
import numpy as np

In [3]:
data = pd.read_csv(r"C:\Users\VY72PC\OneDrive - ING\Documents\STARIMA Project\Data\pm25_weer.csv")
data["Wind"] = np.cos(data["DD"]) * (data["U"] * 1.852)
data

,id,no2,pm10,pm10_cal,pm10_fac,pm10_max,pm10_min,pm25,pm25_cal,pm25_fac,...,jaar,maand,weeknummer,dag,H,T,U,DD,FH,Wind
0,1,NaN,8.195,8.774,1.070000,8.39,8.00,4.000,1.640,0.410000,...,2019,7,28,12,14,173,86,320,60,143.934921
1,4,NaN,17.258,18.165,1.052000,20.10,15.60,11.176,4.256,0.381000,...,2019,7,28,12,14,173,86,320,60,143.934921
2,5,NaN,19.952,20.497,1.027000,24.40,16.50,10.810,3.996,0.370000,...,2019,7,28,12,14,173,86,320,60,143.934921
3,7,NaN,20.278,21.216,1.042000,24.40,16.20,11.510,4.292,0.371000,...,2019,7,28,12,14,173,86,320,60,143.934921
4,10,NaN,18.099,19.186,1.062000,21.90,14.70,8.942,3.471,0.390000,...,2019,7,28,12,14,173,86,320,60,143.934921
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1349808,199,33.4319,24.660,16.796,0.681103,28.50,20.50,11.910,9.877,0.829303,...,2022,10,42,22,23,127,92,140,20,-33.704268
1349809,200,43.1974,0.000,0.000,NaN,0.00,0.00,0.000,0.000,NaN,...,2022,10,42,22,23,127,92,140,20,-33.704268
1349810,212,2.5000,23.074,22.407,0.971261,29.30,18.29,7.887,11.534,1.461850,...,2022,10,42,22,23,127,92,140,20,-33.704268
1349811,213,39.1156,37.694,36.569,0.970257,60.80,24.40,9.182,13.432,1.463180,...,2022,10,42,22,23,127,92,140,20,-33.704268


In [ ]:
data.columns

In [5]:
data.drop(data.iloc[:, 0:7], axis=1, inplace=True)
data.drop(["jaar", "maand", "weeknummer", "#STN", "timestamp", "components", "dag", "tijd", "T", "datum", "H", "T", "U", "weekdag", "FH", "sensortype"], axis=1, inplace=True)
# data

In [6]:
grouped_df = data.groupby(["YYYYMMDD", "tag"])["pm25", "longitude", "latitude", "Wind"].mean().copy().reset_index()

<ipython-input-6-7fe0842539fe>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  grouped_df = data.groupby(["YYYYMMDD", "tag"])["pm25", "longitude", "latitude", "Wind"].mean().copy().reset_index()


In [ ]:
grouped_df.head(5)

In [7]:
Locations = grouped_df["tag"].unique()
LocDict = dict()

for i in range(len(Locations)):
    LocDict[Locations[i]] = (grouped_df[grouped_df.tag == Locations[i]]["latitude"].mean(), grouped_df[grouped_df.tag == Locations[i]]["longitude"].mean())

LocDict

{'Amsterdam': (52.38556547178074, 4.901327224364788),
 'Beverwijk': (52.48636675602225, 4.6585847376827525),
 'Heemskerk': (52.50593616684639, 4.671953092146707),
 'Wijk aan Zee': (52.493443775747366, 4.5982457823463205),
 'Velsen-Noord': (52.47382287197285, 4.647238399148998),
 'Driehuis': (52.44719482753771, 4.636977030738782),
 'IJmuiden': (52.45820717461738, 4.615567828011167),
 'Velsen-Zuid': (52.46234499999992, 4.637255999999981),
 'Zaandam': (52.45848175659799, 4.824912486862416),
 'Koog aan de Zaan': (52.46407900000035, 4.811263999999972),
 'Uithoorn': (52.23809099999961, 4.808228999999986)}

In [8]:
W = np.zeros((11, 11))

for i in range(len(LocDict)):
    for j in range(len(LocDict)):
        if i != j:
            W[i, j] = 1 / great_circle(LocDict[Locations[i]], LocDict[Locations[j]]).km
            
pd.DataFrame(W)

,0,1,2,3,4,5,6,7,8,9,10
0,0.000000,0.050226,0.048748,0.042035,0.050438,0.052102,0.047633,0.050410,0.103925,0.093859,0.056891
1,0.050226,0.000000,0.424320,0.240358,0.627957,0.217623,0.233800,0.329302,0.085581,0.094041,0.033993
2,0.048748,0.424320,0.000000,0.193082,0.253583,0.143926,0.152950,0.185648,0.086023,0.095066,0.032065
3,0.042035,0.240358,0.193082,0.000000,0.251844,0.173214,0.244498,0.229791,0.063146,0.067607,0.031474
4,0.050438,0.627957,0.253583,0.251844,0.000000,0.328792,0.362306,0.692351,0.082260,0.089577,0.035210
5,0.052102,0.217623,0.143926,0.173214,0.328792,0.000000,0.526740,0.593567,0.078144,0.083627,0.038463
6,0.047633,0.233800,0.152950,0.244498,0.362306,0.526740,0.000000,0.649445,0.070501,0.075331,0.036030
7,0.050410,0.329302,0.185648,0.229791,0.692351,0.593567,0.649445,0.000000,0.078608,0.084816,0.036354
8,0.103925,0.085581,0.086023,0.063146,0.082260,0.078144,0.070501,0.078608,0.000000,0.897148,0.040762
9,0.093859,0.094041,0.095066,0.067607,0.089577,0.083627,0.075331,0.084816,0.897148,0.000000,0.039794


In [9]:
data2 = grouped_df.copy()
data2["New"] = data2["YYYYMMDD"].astype(str)
del data2["YYYYMMDD"]

In [10]:
data2.set_index("New", inplace=True)

In [11]:
del data2["latitude"]
del data2["longitude"]
data2

,tag,pm25,Wind
New,,,
20190625,Amsterdam,16.602565,-8.255864
20190625,Beverwijk,18.822000,-17.750135
20190626,Amsterdam,21.974609,-14.046547
20190626,Beverwijk,2.851417,2.343418
20190627,Amsterdam,4.484500,-7.320556
...,...,...,...
20221022,IJmuiden,6.112711,-15.943545
20221022,Uithoorn,7.120000,-16.526770
20221022,Velsen-Noord,6.428708,-16.526770


In [12]:
UniqueNames = data2.tag.unique()

PolDict = {elem : pd.DataFrame() for elem in UniqueNames}
WindDict = {elem : pd.DataFrame() for elem in UniqueNames}

for key in PolDict.keys():
    PolDict[key] = data2[:][data2.tag == key]
    WindDict[key] = data2[:][data2.tag == key]
    del WindDict[key]["pm25"]
    del PolDict[key]["Wind"]

PolDict["Amsterdam"]

,tag,pm25
New,,
20190625,Amsterdam,16.602565
20190626,Amsterdam,21.974609
20190627,Amsterdam,4.484500
20190628,Amsterdam,7.430917
20190629,Amsterdam,7.474792
...,...,...
20221018,Amsterdam,59.648775
20221019,Amsterdam,49.008481
20221020,Amsterdam,57.314282


In [13]:
for key in WindDict:
    PolDict[key].rename(columns={"pm25":key}, inplace=True)
    WindDict[key].rename(columns={"Wind":key}, inplace=True)
    del PolDict[key]["tag"]
    del WindDict[key]["tag"]

WindDict["Amsterdam"]

,Amsterdam
New,
20190625,-8.255864
20190626,-14.046547
20190627,-7.320556
20190628,1.658929
20190629,1.356580
...,...
20221018,57.190670
20221019,7.033693
20221020,8.239286


In [ ]:
# for key in tqdm(DataFrameDict):
#     plt.figure(figsize=(16, 8))
#     plt.plot(DataFrameDict[key].pm25, label = key)
# plt.show

In [14]:
df_pol = pd.DataFrame(PolDict["Amsterdam"].copy())
df_wind = pd.DataFrame(WindDict["Amsterdam"].copy())

for key in PolDict:
    df_pol = df_pol.combine_first(PolDict[key])
    df_wind = df_wind.combine_first(PolDict[key])

df_wind

,Amsterdam,Beverwijk,Driehuis,Heemskerk,IJmuiden,Koog aan de Zaan,Uithoorn,Velsen-Noord,Velsen-Zuid,Wijk aan Zee,Zaandam
New,,,,,,,,,,,
20190625,-8.255864,18.822000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20190626,-14.046547,2.851417,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20190627,-7.320556,4.010603,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20190628,1.658929,5.533953,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20190629,1.356580,9.006055,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
20221018,57.190670,44.425756,7.785098,3.995484,4.095699,NaN,5.980941,5.637824,NaN,117.669196,7.467459
20221019,7.033693,45.131791,11.062278,4.602050,5.595226,NaN,5.646750,6.915667,NaN,117.688819,5.922350
20221020,8.239286,55.310167,36.090788,15.024389,22.693654,NaN,27.862864,25.996136,NaN,130.220561,23.761873


In [15]:
for column in df_pol:
    mean_value = (df_pol[column].mean(), df_wind[column].mean())
    df_pol[column].fillna(value=mean_value[0], inplace = True)
    df_wind[column].fillna(value=mean_value[1], inplace = True)
    
df_wind

,Amsterdam,Beverwijk,Driehuis,Heemskerk,IJmuiden,Koog aan de Zaan,Uithoorn,Velsen-Noord,Velsen-Zuid,Wijk aan Zee,Zaandam
New,,,,,,,,,,,
20190625,-8.255864,18.822000,12.833248,17.620544,10.040674,15.217022,29.655720,10.477617,9.501286,24.129598,12.378897
20190626,-14.046547,2.851417,12.833248,17.620544,10.040674,15.217022,29.655720,10.477617,9.501286,24.129598,12.378897
20190627,-7.320556,4.010603,12.833248,17.620544,10.040674,15.217022,29.655720,10.477617,9.501286,24.129598,12.378897
20190628,1.658929,5.533953,12.833248,17.620544,10.040674,15.217022,29.655720,10.477617,9.501286,24.129598,12.378897
20190629,1.356580,9.006055,12.833248,17.620544,10.040674,15.217022,29.655720,10.477617,9.501286,24.129598,12.378897
...,...,...,...,...,...,...,...,...,...,...,...
20221018,57.190670,44.425756,7.785098,3.995484,4.095699,15.217022,5.980941,5.637824,9.501286,117.669196,7.467459
20221019,7.033693,45.131791,11.062278,4.602050,5.595226,15.217022,5.646750,6.915667,9.501286,117.688819,5.922350
20221020,8.239286,55.310167,36.090788,15.024389,22.693654,15.217022,27.862864,25.996136,9.501286,130.220561,23.761873


In [ ]:
VARModel = VAR(df_pol).fit()

In [ ]:
WY = pd.DataFrame(np.matmul(df_pol.to_numpy(), W))

i = 0
for key in PolDict:
    WY.rename(columns={i:f'{key}'}, inplace=True)
    i += 1

WY

In [ ]:
SVAR = VAR(WY).fit()
SVAR.summary()

In [16]:
WW = list()

for i in range(len(df_wind)):
    wind = np.zeros((len(df_wind.columns), len(df_wind.columns)))
    
    for j in range(len(df_wind.columns)):

        if df_wind.iloc[i, j] != 0:
            wind[:, j] = W[:, j] + 1 / df_wind.iloc[i, j]
        else:
            wind[:, j] = W[:, j]

    WW.append(wind)

In [17]:
WWY = np.zeros((len(df_pol), len(df_pol.columns)))

for i in range(len(df_pol)):
    WWY[i, :] = np.matmul(df_pol.iloc[i, :].to_numpy(), WW[i])

WWY = pd.DataFrame(WWY)

# i = 0
# for key in PolDict:
#     WWY.rename(columns={i:f'{key}'}, inplace=True)
#     i += 1

WWY

,0,1,2,3,4,5,6,7,8,9,10
0,-11.965021,43.436811,40.737557,35.204544,64.385599,49.888109,38.831773,68.873574,51.293132,27.397866,20.607236
1,-3.160338,92.743279,33.396931,30.990210,53.572160,45.995939,34.996354,62.873686,49.369164,25.960965,19.513790
2,-11.773827,70.897511,31.763650,29.606825,51.791435,44.263742,33.883581,60.815085,45.931899,23.751565,17.238903
3,102.165642,61.533730,32.901963,30.350494,53.341810,45.042506,34.530809,61.891858,46.838912,24.356611,17.819392
4,125.731726,51.140642,34.651358,31.386425,55.874526,46.031458,35.463236,63.373012,47.510669,24.832960,18.223946
...,...,...,...,...,...,...,...,...,...,...,...
1200,21.724686,34.865371,76.112255,100.339636,125.257065,51.795902,85.412994,96.667865,148.804572,24.567674,49.185574
1201,56.343829,36.888115,65.355851,90.838123,107.526228,52.652303,88.915551,87.912760,147.694058,24.099965,57.781990
1202,75.158738,75.883263,71.764820,81.848303,115.401550,100.468990,84.348969,102.868280,183.701276,35.324027,33.306510
1203,1.110264,57.236656,67.941068,87.661026,108.873984,80.271160,77.751593,93.123855,170.129193,31.973472,42.145197


In [18]:
SWVAR = VAR(WWY).fit()
SWVAR.summary()

  Summary of Regression Results   
Model:                         VAR
Method:                        OLS
Date:           Mon, 19, Dec, 2022
Time:                     10:51:49
--------------------------------------------------------------------
No. of Equations:         11.0000    BIC:                    68.1168
Nobs:                     1204.00    HQIC:                   67.7687
Log likelihood:          -59330.6    FPE:                2.18896e+29
AIC:                      67.5584    Det(Omega_mle):     1.96272e+29
--------------------------------------------------------------------
Results for equation 0
           coefficient       std. error           t-stat            prob
------------------------------------------------------------------------
const        27.948458        27.673061            1.010           0.313
L1.0          0.001024         0.028948            0.035           0.972
L1.1         -0.289760         0.589716           -0.491           0.623
L1.2          0.625127 

In [19]:
for key in PolDict:
    R2 = skm.r2_score(SWVAR.fittedvalues[key] + SWVAR.resid[key], SWVAR.fittedvalues[key])
    print(F'The R-Squared of {key} is: {R2*100:.2f}%')

    # y = SWVAR.fittedvalues['foobar'] + SWVAR.resid['foobar']
    # R2 = 1 - np.sum(SWVAR.resid['foobar'].values ** 2) / np.sum((y.values - y.mean()) ** 2)

KeyError: 'Amsterdam'